In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import csv
import re
from datasets import Dataset
from datasets import load_from_disk
from bertopic.representation import VisualRepresentation
import pandas as pd
from sklearn.feature_extraction import text

In [11]:
def remove_links(text):
    # Remove links starting with 'https' or 'http'
    text_without_links = re.sub(r'https?\S+', '', text)
    return text_without_links

def extract_text_from_csv(csv_file):
    texts = []                                              # Create an empty list to store texts
    ids = []                                                # Create an empty list to store tweet IDs
    with open(csv_file, 'r', encoding='utf-8-sig') as file: # Open the CSV file in read mode
        reader = csv.DictReader(file)                       # Create a CSV reader object
        for row in reader:                                  # Iterate through each row in the CSV file
            text = row['text']                              # Get the value of the 'text' column in the current row
            id = row['tweet_id']                            # Get the value of the 'tweet_id' column in the current row
            url = row['url']
            if text != '':                                  # Check if the text is not empty
                if url != '':                               # Check if url is not empty
                    cleaned = remove_links(text)            # Clean the text of links
                    texts.append(cleaned)                   # Add the text to the 'texts' list
                    ids.append(id)                          # Add the tweet ID to the 'ids' list
    return texts, ids                                       # Return the 'texts' list and the 'ids' list as a tuple

In [112]:
#TEXTS, IDS = extract_text_from_csv('twitter_BC_AB_wildfires_apr-jun2023_5,478.csv')
#TEXTS, IDS = extract_text_from_csv('ottawa_tornado_2018~10,009.csv')
#TEXTS, IDS = extract_text_from_csv('ottawa_storm_2022~9945.csv')
TEXTS, IDS = extract_text_from_csv('NS_wildfire_2023~10,017.csv')
print(len(TEXTS))

4756


In [118]:
#dataset_name = 'twitter_BC_AB_wildfires_apr-jun2023_5,478(dataset)'
#dataset_name = 'ottawa_tornado_2018~10,009(dataset)'
#dataset_name = 'ottawa_storm_2022~9945(dataset)'
dataset_name = 'NS_wildfire_2023~10,017(dataset)'
#model_name = 'twitter_BC_AB_wildfires_apr-jun2023_5,478(text_model3)'
#model_name = 'ottawa_tornado_2018~10,009(text_model3)'
#model_name = 'ottawa_storm_2022~9945(text_model3)'
model_name = 'NS_wildfire_2023~10,017(text_model3)'

In [114]:
loaded_dataset = load_from_disk(dataset_name)
docs = loaded_dataset["text"]
print(len(docs))

5737


In [ ]:
default_stop_words = text.ENGLISH_STOP_WORDS
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=default_stop_words) #"english")

topic_model = BERTopic(
    #verbose=True,
    vectorizer_model = vectorizer_model,
    language = 'english', 
    calculate_probabilities = True,
    nr_topics = 35
    )

In [41]:
topics, probs = topic_model.fit_transform(docs)

In [43]:
topic_model.save(model_name, serialization="safetensors")

In [119]:
topic_model = BERTopic.load(model_name)

In [120]:
topic_model.get_topic_info().drop('Representative_Docs',1)

,Topic,Count,Name,Representation
0,-1,2066,-1_wildfires_wildfire_smoke_amp,"[wildfires, wildfire, smoke, amp, canada, albe..."
1,0,767,0_smoke_wildfires_alberta_canada,"[smoke, wildfires, alberta, canada, wildfire, ..."
2,1,702,1_wildfire_wildfires_amp_new,"[wildfire, wildfires, amp, new, home, forest, ..."
3,2,695,2_sk wildfire_update detected_info sk_wildfire...,"[sk wildfire, update detected, info sk, wildfi..."
4,3,237,3_halifax_tantallon_nsfire_novascotia,"[halifax, tantallon, nsfire, novascotia, nova,..."
5,4,112,4_wildfires_climatechange_spain_hectares,"[wildfires, climatechange, spain, hectares, cl..."
6,5,110,5_alberta_canada_wildfires_acres,"[alberta, canada, wildfires, acres, province, ..."
7,6,99,6_water_drone_hotshots_aircraft,"[water, drone, hotshots, aircraft, hotshot, av..."
8,7,88,7_air_smoke_wildfire smoke_health,"[air, smoke, wildfire smoke, health, airqualit..."
9,8,85,8_nsfire_nsfires_novascotia nsfires_shelburne,"[nsfire, nsfires, novascotia nsfires, shelburn..."


In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
hier = topic_model.hierarchical_topics(TEXTS)
tree = topic_model.get_topic_tree(hier)
print(tree)

In [23]:
topic_distr, _ = topic_model.approximate_distribution(TEXTS)

In [ ]:
topic_model.visualize_distribution(topic_distr[1])

In [10]:
topic_model.visualize_hierarchy()

In [14]:
topic_model.merge_topics(TEXTS, topics_to_merge=[19,21])

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.get_topics()